In [41]:
import pandas as pd
base_forms = pd.read_excel('base-forms.xlsx') 
base_forms = base_forms.fillna('')
import re

In [42]:
base_forms.head()

,p-3a,p-3b,p-2,p-1,p0a,p0b,p+1,p+2,p+3,p+4,transitivity,infinitive_UR,infinitive_meaning
0,a,,,,ger,gerd,,i,,,intrans,a-ger-i-ast-*an*,"to be bought, to be gotten"
1,a,,,,ger,gerd,,,,,trans,a-gerd-*an*,"to buy, to get"
2,,be,,,andʌd͡ʒ,andʌt,,i,,,intrans,be-andʌd͡ʒ-i-ast-*an*,to be hung
3,,be,,,andʌd͡ʒ,andʌt,,,,,trans,be-andʌt-*an*,to hang
4,,be,,,and͡ʒan,and͡ʒand,,i,,,intrans,be-and͡ʒan-i-ast-*an*,"to be broken into peices (bread, etc.)"


In [43]:
def stressed(position):
        return "*" + position + "*" # the stressed morpheme will be represented in the UR with embedding asteriks

In [57]:


def verb_generator(row, tense, modality, aspect, polarity, person): # the variables chosen by user: now iterated over by the program  
    pm3a = row['p-3a']
    pm3b = row['p-3b']
    pm2 = row['p-2']
    pm1 = row['p-1']
    p0a = row['p0a']
    p0b = row['p0b']
    pp1 = row['p+1']
    pp2 = row['p+2']
    pp3 = row['p+3']
    pp4 = row['p+4']
    transitivity = row['transitivity'] 
  
    # causative / non-causative
    if pp1 == 'den':
        causation = 'causative'
    else:
        causation = 'non-causative'

    #active/passive:
    if pp2 == 'i':
        voice = 'passive'
    else:
        voice = 'active'

    tense_options = ['present','past']
    
    #start with present tense logic:
    if tense == 'present':

        p0b = '' #present verbs only use a neutral stem: their past stem is always empty
        if  pp2 == '': #imperative modality is incompatiable with 'i' in pp2 (passive voice)
            modality_options = ['imperative', 'subjunctive', 'indicative']
            if modality not in modality_options:
                return
        elif pp2 == 'i':
            modality_options = ['subjunctive', 'indicative']
            if modality not in modality_options:
                return
        else: 
            raise Exception('present-modality options error')
        #setting up person options for modality values:
        if modality == 'imperative':  
            person_options = ['2s','2p']
            if person not in person_options:
                return
            #imperative verbs are only made for second person
        elif  modality == 'subjunctive' or modality == 'indicative':
            person_options = ['1s', '2s', '3sm', '3sf', '1p', '2p', '3p']
            if person not in person_options:
                return
        else: 
            raise Exception('present- person selections error')

        #now, logic for each modality opetion:
        if modality == 'imperative':
            if  person == '2s':
                pp4 = '∅' #because this program produces underlying morphological forms, null morphemes are written down
            elif person == '2p':
                pp4 = 'ʌ'
            else:
                raise Exception('present - imperative - pp4 error')

        elif modality == 'indicative' or modality == 'subjunctive': #non-imperative verbs are inflected for all persons
            if person == '1s':
                pp4 = 'em(ɛ)'
            elif person == '2s':
                pp4 = 'i'
            elif person == '3sm':
                pp4 = 'e'
            elif person == '3sf':
                pp4 = 'iɛ'
            elif person == '1p':
                pp4 = 'om(ɛ)'
            elif person == '2p':
                pp4 = 'ʌ'
            elif person == '3p':
                pp4 = 'endɛ'

        if modality == 'indicative':
            aspect_options = ['imperfective']
            if aspect not in aspect_options:
                return
            if aspect == 'imperfective':
                pm2 = 'me'
                pm3b = ''

        elif modality == 'subjunctive' or modality == 'imperative':
            aspect_options = ['neutral']
            if aspect not in aspect_options:
                return
            if aspect == 'neutral':
                pm2 = ''

        polarity_options = ['negative', 'affirmative']
        if polarity not in polarity_options:
                return

        if polarity == 'negative':
            if modality == 'imperative':
                pm1 = 'ma'
            elif modality == 'indicative' or modality == 'subjunctive':
                pm1 = 'ne'
            pm1 = stressed(pm1)
            pm3b = '' #so the content of pm3b is ovverriden due to negation      
        elif polarity == 'affirmative':
            if modality == 'imperative' or modality == 'subjunctive':
                if pm3a != '': #if a verb has a preverb, pv gets stressed
                    pm3a = stressed(pm3a)
                elif pm3b != '': 
                    pm3b = stressed(pm3b)   
                elif pm3a == '' and pm3b == '':
                    p0a = stressed(p0a) #stem gets stressed instead
            elif modality == 'indicative':
                pp4 = stressed(pp4) # in indicative verbs, AGR is stressed


    elif tense == 'past':
        person_options = ['1s', '2s', '3sm', '3sf', '1p', '2p', '3p'] #past tense verbs are inflected for all persons
        if person not in person_options:
            return
        if pp1 == 'den' or pp2 == 'i': # if one of these positions is filled
            pp3 = 'ast' # then there needs to be 'ast' in pp3
            p0b = '' # present stem is used
        elif pp1 == '' and pp2 == '' and p0b == '': #if both of pp1 and pp2 are empty and verb lacks a past stem
            pp3 = 'ast' #the past tense has to be marked with 'ast'
            p0b = ''
        else: # then use past stem
            p0a = ''

        modality_options = ['subjunctive', 'indicative']
        if modality not in modality_options:
                return

        if modality == 'indicative':
            aspect_options = ['perfect', 'pluperfect', 'neutral', 'imperfective']
            if aspect not in aspect_options:
                return
        elif modality == 'subjunctive':
            aspect_options = ['perfect'] #imperfective aspect and subjunctive modality are incompatible
            if aspect not in aspect_options:
                return
        #1
        if modality == 'subjunctive' and transitivity == 'intrans':
            if person == '1s':
                pp4 = 'abim(ɛ)'
            elif person == '2s':
                pp4 = 'abiʃ(ɛ)'
            elif person == '3sm':
                pp4 = 'abe'
            elif person == '3sf':
                pp4 = 'abiɛ'
            elif person == '1p':
                pp4 = 'abimon'
            elif person == '2p':
                pp4 = 'abion'
            elif person == '3p':
                pp4 = 'abindɛ'
        #2
        elif modality == 'subjunctive' and transitivity == 'trans':
            if person == '1s':
                pp4 = 'abim(ɛ)'
            elif person == '2s':
                pp4 = 'abi'
            elif person == '3sm':
                pp4 = 'abiʃ(ɛ)'
            elif person == '3sf':
                pp4 = 'abiʃ(ɛ)'
            elif person == '1p':
                pp4 = 'abimon'
            elif person == '2p':
                pp4 = 'abion'
            elif person == '3p':
                pp4 = 'abiʃon'
    #3
        elif modality == 'indicative' and aspect == 'perfect' and transitivity == 'intrans':

            if person == '1s':
                pp4 = 'aim(ɛ)'
            elif person == '2s':
                pp4 = 'aiʃ(ɛ)'
            elif person == '3sm':
                pp4 = 'i'
            elif person == '3sf':
                pp4 = 'iʌ'
            elif person == '1p':
                pp4 = 'aimon'
            elif person == '2p':
                pp4 = 'aion'
            elif person == '3p':
                pp4 = 'indɛ'
        #4    
        elif modality == 'indicative' and aspect == 'perfect' and transitivity == 'trans':

            if person == '1s':
                pp4 = 'im(ɛ)'
            elif person == '2s':
                pp4 = 'i'
            elif person == '3sm':
                pp4 = 'iʃ(ɛ)'
            elif person == '3sf':
                pp4 = 'iʃ(ɛ)'
            elif person == '1p':
                pp4 = 'imon'
            elif person == '2p':
                pp4 = 'ion'
            elif person == '3p':
                pp4 = 'iʃon'      

        #5    
        elif modality == 'indicative' and aspect == 'pluperfect' and transitivity == 'intrans':  

            if person == '1s':
                pp4 = 'avim(ɛ)'
            elif person == '2s':
                pp4 = 'aviʃ(ɛ)'
            elif person == '3sm':
                pp4 = 'ave'
            elif person == '3sf':
                pp4 = 'aviɛ'
            elif person == '1p':
                pp4 = 'avimon'
            elif person == '2p':
                pp4 = 'avion'
            elif person == '3p':
                pp4 = 'avindɛ'  

        #6
        elif modality == 'indicative' and aspect == 'pluperfect' and transitivity == 'trans':  

            if person == '1s':
                pp4 = 'avim(ɛ)'
            elif person == '2s':
                pp4 = 'avi'
            elif person == '3sm':
                pp4 = 'aviʃ(ɛ)'
            elif person == '3sf':
                pp4 = 'aviʃ(ɛ)'
            elif person == '1p':
                pp4 = 'avimon'
            elif person == '2p':
                pp4 = 'avion'
            elif person == '3p':
                pp4 = 'aviʃon'  

    #7
        elif transitivity == 'intrans': 

            if person == '1s':
                pp4 = 'im(ɛ)'
            elif person == '2s':
                pp4 = 'iʃ(ɛ)'
            elif person == '3sm':
                pp4 = '∅'
            elif person == '3sf':
                pp4 = 'ɛ'
            elif person == '1p':
                pp4 = 'imon'
            elif person == '2p':
                pp4 = 'ion'
            elif person == '3p':
                pp4 = 'indɛ'
        #8        
        elif transitivity == 'trans': 

            if person == '1s':
                pp4 = 'em(ɛ)'
            elif person == '2s':
                pp4 = 'i'
            elif person == '3sm':
                pp4 = 'eʃ(ɛ)'
            elif person == '3sf':
                pp4 = 'eʃ(ɛ)'
            elif person == '1p':
                pp4 = 'emon'
            elif person == '2p':
                pp4 = 'ion'
            elif person == '3p':
                pp4 = 'eʃon'


        else:
            raise Exception('past- agreement error')

        if aspect == 'imperfective':
            pm2 = 'me'
            # imperfective verbs cannot have their pm3b filled
            pm3b = '' 

        polarity_options = ['negative', 'affirmative']
        if polarity not in polarity_options:
            return

        if polarity == 'negative':
            pm1 = 'ne'
            pm1 = stressed(pm1)
            #negative verbs cannot have their pm3b filled
            pm3b = ''

        else: #polarity is affirmative
            if aspect == 'perfect' or aspect == 'pluperfect' or (modality == 'subjunctive' and aspect is not 'neutral') :
                pp4 = stressed(pp4)
            else: # nuetral aspect is marked by stress, in past tense verbs
                if pp3 == 'ast':
                    pp3 = stressed(pp3)
                elif pp3 == '':
                    p0b = stressed(p0b)




    pieces = [pm3a, pm3b, pm2, pm1, p0a, p0b, pp1, pp2, pp3, pp4]
    wanted_pieces = []
    for i in pieces:
        if i != '':
            wanted_pieces.append(i)
    generated_verb = '-'.join(wanted_pieces)
    return(generated_verb, voice, transitivity, causation)


# next is to iterate over all verbs and put them into a single dataframe


<>:301: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:301: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/cl/6732l4td4930rpqrp4rpmpt80000gn/T/ipykernel_28580/502307373.py:301: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if aspect == 'perfect' or aspect == 'pluperfect' or (modality == 'subjunctive' and aspect is not 'neutral') :


In [58]:
tense_options_all = ['present', 'past']
aspect_options_all = ['imperfective', 'neutral', 'perfect', 'pluperfect']
modality_options_all = ['indicative', 'imperative', 'subjunctive']
person_options_all = ['1s', '2s', '3sm', '3sf', '1p', '2p', '3p']
polarity_options_all = ['negative', 'affirmative']
voice_options_all = ['passive', 'active']
causation_options_all = ['causative', 'non-causative']

In [59]:
verbs = []
for i, row in base_forms.iterrows():
    for tense in tense_options_all:
        for modality in modality_options_all:
            for aspect in aspect_options_all:
                for polarity in polarity_options_all:
                    for person in person_options_all:
                        results = verb_generator(row, tense, modality, aspect, polarity, person)
                        if results: #exists
                            generated_verb, voice, transitivity, causation = results
                            verbs.append((tense, modality, aspect, causation, polarity, person, voice, transitivity, generated_verb))

In [60]:
all_verbs = pd.DataFrame(verbs).reset_index(drop = True)

In [61]:
all_verbs

,0,1,2,3,4,5,6,7,8
0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ)
1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i
2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e
3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ
4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ)
...,...,...,...,...,...,...,...,...,...
34475,past,subjunctive,perfect,non-causative,affirmative,3sm,active,trans,ʌ-χʌst-*abiʃ(ɛ)*
34476,past,subjunctive,perfect,non-causative,affirmative,3sf,active,trans,ʌ-χʌst-*abiʃ(ɛ)*
34477,past,subjunctive,perfect,non-causative,affirmative,1p,active,trans,ʌ-χʌst-*abimon*
34478,past,subjunctive,perfect,non-causative,affirmative,2p,active,trans,ʌ-χʌst-*abion*


In [62]:
len(verbs)

34480

In [63]:
all_verbs = all_verbs.rename(columns = {0:'Tense', 1:'Modality', 2:'Aspect', 3: 'Causation', 4:'Polarity', 5: 'Person', 6: 'Voice', 7: 'Transitivity',8: 'Generated Verb-UR'})

In [64]:
all_verbs.to_csv('./all_verbs.csv', encoding = 'utf-8-sig')

In [65]:
all_verbs.head()

,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR
0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ)
1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i
2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e
3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ
4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ)


In [66]:
all_verbs[all_verbs['Generated Verb-UR'] == 'ʌ-der-den-i-*ast*-∅']

,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR
27123,past,indicative,neutral,causative,affirmative,3sm,passive,intrans,ʌ-der-den-i-*ast*-∅


In [67]:
all_verbs[all_verbs['Generated Verb-UR'] == 'ʌ-*rit*-em(ɛ)']

,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR
30631,past,indicative,neutral,non-causative,affirmative,1s,active,trans,ʌ-*rit*-em(ɛ)
